# shst_01_extract_short_segments

A quirk of the SharedStreets referencing system that had to be addressed was that intersections between two adjacent roadbeds were represented as a short segment line instead of an intersection point. <br>
For this reason, in this notebook, segment where were less than 100ft and lion node points were mapped on was identified as node points (we named this as 'short segment')

**Before running this notebook, mapping the lion node points into Sharedstreet geometry with 'SharedStreets API' is required.**

## 1. Extract Short segment

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# make sure that you mapped the lion node points using Sharedstreet API
# import the lion node point dataset that mapped on the sharedstreets api
gdf_lion_matched = gpd.read_file('../data/sharedstreets_results/lion_node/lion_node.matched.geojson')

In [3]:
gdf_lion_matched.head(3)

,score,location,referenceLength,geometryId,referenceId,direction,bearing,sideOfStreet,interceptAngle,geometry
0,0.052116,96.843546,96.91,04e535b2594119b4e9d4cfc368ba127c,b4b87ba043856810117dabd942a28dfa,backward,261.200149,right,90.208016,POINT (-74.25422 40.50617)
1,2.137940,0.154738,40.88,9b44f18a30f0dc2131f392909ee36a0e,daaf02da8f37d979848cdbd49f127ba7,forward,347.958348,right,89.996935,POINT (-74.25412 40.50581)
2,2.764782,246.372624,329.59,6c754d6c85315a5a2398c0c8964e9998,b9a7a78847e272d1a83dfd7aa6b2b428,forward,349.994421,left,270.001425,POINT (-74.25314 40.50477)


In [4]:
# make sure that you run the 'shst_00_processing_sharedstreets_geometry' notebook to get the 'shst_segment.shp'
# import the sharedstreet segments
gdf_shst_segment = gpd.read_file('../data/sharedstreets_geometry/segment/shst_segment.shp')

In [5]:
gdf_shst_segment.head(3)

,id,fromInters,toIntersec,forwardRef,backRefere,roadClass,length,geometry
0,db6792075ebbddc84479fda26174ca30,374b01a56e64379b8d7198962eaede90,2922a5babc5f921116a9fed4131a5bb1,48b7ab8e4cbafb2c1893cd682ded6704,a8475c8bd67f9e0ec8ce6a404aae41c1,Residential,299.421958,"LINESTRING (-73.91694 40.64668, -73.91625 40.6..."
1,42ccdc2b9ebc38f98c22bb0035045628,37db438d57f16f92e5ba91f1ad1793bb,374b01a56e64379b8d7198962eaede90,febaf06db79d8a16588d1c387a62fdb2,9db38906c3d8ae5df463e297be4e2b9b,Residential,256.577253,"LINESTRING (-73.91765 40.64623, -73.91732 40.6..."
2,84afb6627019b793945a7aab1feefe77,374b01a56e64379b8d7198962eaede90,5b6e4972c82ad4eb6d24c17b94b33b59,3f53ec240fc39c6b6810243b5b6fc830,fbbb71d35b794421e030d3ec9e1dcede,Residential,264.356932,"LINESTRING (-73.91694 40.64668, -73.91662 40.6..."


In [6]:
# extracted id of sharedstreets segments that matched with lion node points
list_lion_matched_shst_id = gdf_lion_matched['geometryId'].unique().tolist()

In [7]:
# I found one segment that was missed
list_lion_matched_shst_id.append('29bf447ea077f90a45ae92eadc8fac05') 

In [8]:
# filter the segment that were matched with lion node points
gdf_shst_short_segment = gdf_shst_segment.loc[gdf_shst_segment['id'].isin(list_lion_matched_shst_id)]

In [9]:
# filter the segment that were less than 100ft
gdf_shst_short_segment = gdf_shst_short_segment.loc[gdf_shst_short_segment['length']<100]

In [10]:
# export the dataset
gdf_shst_short_segment.to_file('../data/sharedstreets_geometry/short_segment/shst_short_segment.shp')

In [11]:
# export the dataset
gdf_shst_short_segment.to_file('../data/sharedstreets_geometry/short_segment/shst_short_segment.geojson', driver='GeoJSON')

Here, we will calculate the centroid of the 'short segment'. This is needed to draw a map in the dashboard

In [12]:
# calculate centroid of the short segments
gdf_shst_short_segment['centroid'] = gdf_shst_short_segment['geometry'].centroid

In [14]:
# set the geometry as 'centroid'
gdf_shst_short_segment = gdf_shst_short_segment.drop('geometry', axis=1)
gdf_shst_short_segment['geometry'] = gdf_shst_short_segment['centroid']
gdf_shst_short_segment = gdf_shst_short_segment.drop('centroid', axis=1)

In [16]:
# export the dataset
gdf_shst_short_segment.to_file('../data/sharedstreets_geometry/short_segment/shst_short_segment_centroid.shp')

In [17]:
# export the dataset
gdf_shst_short_segment.to_file('../data/sharedstreets_geometry/short_segment/shst_short_segment_centroid.geojson', driver='GeoJSON')

## 2. Identify sharedstreets segments which are not assigned as 'short segment'

This part for identifying the segments which are not identified as the 'Short segment'.

In [23]:
# get segments which is not the short segments
list_short_segment = gdf_shst_short_segment['id'].unique().tolist()
gdf_shst_segment_filtered = gdf_shst_segment.loc[~gdf_shst_segment['id'].isin(list_short_segment)]

In [27]:
gdf_shst_segment_filtered.to_file('../data/sharedstreets_geometry/segment_filtered/shst_segment_filtered.shp')

In [26]:
gdf_shst_segment_filtered.to_file('../data/sharedstreets_geometry/segment_filtered/shst_segment_filtered.geojson', driver='GeoJSON')

In [28]:
gdf_shst_segment_filtered[['id','geometry']].to_file('../data/sharedstreets_geometry/segment_filtered/shst_segment_filtered_simplified.geojson', driver='GeoJSON')

## 3. Identify sharedstreets nodes that are not linked with the short segments

This part for identifying the node points which are not connected as the 'Short segment'. It is important to avoid duplicates when we count the number of crashes per node point and short segment

In [29]:
# extract the list of node points that are linked with the short segments
list_from_intersection = gdf_shst_short_segment['fromInters'].unique().tolist()
list_to_intersection = gdf_shst_short_segment['toIntersec'].unique().tolist()

In [30]:
# make sure that you run the 'shst_00_processing_sharedstreets_geometry' notebook to get the 'shst_node.shp'
# import the sharedstreet nodes
gdf_shst_node = gpd.read_file('../data/sharedstreets_geometry/node/shst_node.shp')

In [31]:
# get nodes that are not linked with the short segments
gdf_shst_node_filtered = gdf_shst_node.loc[~gdf_shst_node['node_id'].isin(list_from_intersection)]
gdf_shst_node_filtered = gdf_shst_node_filtered.loc[~gdf_shst_node_filtered ['node_id'].isin(list_to_intersection)]

In [32]:
gdf_shst_node_filtered.to_file('../data/sharedstreets_geometry/node_filtered/shst_node_filtered.shp')

In [33]:
gdf_shst_node_filtered.to_file('../data/sharedstreets_geometry/node_filtered/shst_node_filtered.geojson', driver='GeoJSON')